# Dynamic Attributes and Properties
Data attributes and methods are together known as 'attributes' in Python. A method is an attribute that is callable.

Properties:  can replace a public data attribute with accessor methods (getter/setter) without changing the class interface. 

Interpreter calls special methods such as __getatrr__ and __setattr__ to evaluate attribute access using dot notation. 

User defined classes implementing __getattr__ can implement 'virtual attrs'; create vars on the fly when someone tries to read a non-existent attr, like obj.no_such_attribute.

In [3]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # issue a warning if a new DL will be made
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
        # with using two context managers to read remove file and save it
            local.write(remote.read())
    
    with open(JSON) as fp:
        return json.load(fp)  # json.load parses a JSON file and returns
    # native Python objects. In this feed, we have the types:
    # dict, list, str, int

In [4]:
feed = load()

/Users/tony/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json


In [5]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [6]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

## Exploring JSON-Like Data with Dynamic Attributes

Uses a recursive and read-only FrozenJSON class.

Key is the __getattr__ method --> called by interpreter when the usual process fails to retrieve an attribute. (i.e. - named attribute is not found in this instance, nor in the class or its superclasses)



In [7]:
from collections import abc

class FrozenJSON:
    """A read only facade for navigating a JSON-like object 
       using attribute notation
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping)  # build a dict from the mapping arg
        # This serves two purposes: ensures we got a dict (or something
        # that can be converted to one) and makes a copy for safety
        
    def __getattr__(self, name):  # getarr is only called when
        # there's no attribute with that name
        if hasattr(self.__data, name):
            return getattr(self.__data, name)  # if name matches an attr
        # of the instanec __data, return that. This is how calls
        # to methods like keys are handled
        else:
            return FrozenJSON.build(self.__data[name]) # otherwise, fetch
        # the item with the key name from self.__data and return
        # the result of calling FrozenJSON.build() on that
        # --> this is where a KeyError may occur. If self.__data[name]
        # should be handled an AttributeError raised instead, because that
        # is what's expected from __getattr__. 
    
    @classmethod
    def build(cls, obj):  # this is an alternate constructor, common
        # use for a classmethod
        if isinstance(obj, abc.Mapping):  # if obj is a mapping build
            # then build FrozenJSON with it
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            # if it is a mutable seq, it must be a list (bc data src
            # is json, & it must be a list or dict), so we build a list
            # by passing every item in obj recursively to .build()
            return [cls.build(item) for item in obj]
        else:  # otherwise, return the item as it is
            return obj

No caching or transformation of the original feed is done. As the feed is traversed, the nested data structures are converted again and again into FrozenJSON. 

Biggest potential issue:  Keys in the original data strucutre may not be suitable attribute names. Next section addresses this.